<a href="https://colab.research.google.com/github/LaxmiSahithiNagarapu/LaxmiSahithi_INFO5731_Spring_2023/blob/main/Nagarapu_In_class_exercise_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fourth in-class-exercise (40 points in total, 03/28/2022)**

Question description: Please use the text corpus you collected in your last in-class-exercise for this exercise. Perform the following tasks:

## (1) (10 points) Generate K topics by using LDA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [ ]:
# Write your code here

# Importing modules
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd

papers = pd.read_csv('papers.csv')
papers.head()



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


In [ ]:
# Remove the columns
papers = papers.drop(columns=['id', 'event_type', 'pdf_name'], axis=1).sample(100)

papers.head()

,year,title,abstract,paper_text
2529,2007,Modeling homophily and stochastic equivalence ...,This article discusses a latent variable model...,Modeling homophily and stochastic equivalence ...
3003,2009,Noisy Generalized Binary Search,This paper addresses the problem of noisy Gene...,Noisy Generalized Binary Search\n\nRobert Nowa...
3871,2012,Recovery of Sparse Probability Measures via Co...,We consider the problem of cardinality penaliz...,Recovery of Sparse Probability Measures via Co...
889,2000,Incremental and Decremental Support Vector Mac...,Abstract Missing,Incremental and Decremental Support Vector\nMa...
2421,2007,Online Linear Regression and Its Application t...,We provide a provably efficient algorithm for ...,Online Linear Regression and Its Application t...


In [ ]:
# Load the regular expression library
import re
# Remove punctuation
papers['paper_text_processed'] = \
papers['paper_text'].map(lambda x: re.sub('[,\.!?]', '', x))
# Convert the titles to lowercase
papers['paper_text_processed'] = \
papers['paper_text_processed'].map(lambda x: x.lower())
# Print out the first rows of papers
papers['paper_text_processed'].head()

2529    modeling homophily and stochastic equivalence ...
3003    noisy generalized binary search\n\nrobert nowa...
3871    recovery of sparse probability measures via co...
889     incremental and decremental support vector\nma...
2421    online linear regression and its application t...
Name: paper_text_processed, dtype: object

In [ ]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))
# remove stop words
data_words = remove_stopwords(data_words)
print(data_words[:1][0][:30])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['modeling', 'homophily', 'stochastic', 'equivalence', 'symmetric', 'relational', 'data', 'peter', 'hoff', 'departments', 'statistics', 'biostatistics', 'university', 'washington', 'seattle', 'wa', 'hoff', 'abstract', 'article', 'discusses', 'latent', 'variable', 'model', 'inference', 'prediction', 'symmetric', 'relational', 'data', 'model', 'based']


In [ ]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 5), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 6), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 5), (22, 2), (23, 1), (24, 4), (25, 1), (26, 1), (27, 3), (28, 1), (29, 1)]


In [ ]:
from pprint import pprint
# number of topics
num_topics = 10
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.007*"model" + 0.007*"data" + 0.006*"learning" + 0.005*"algorithm" + '
  '0.005*"function" + 0.004*"set" + 0.004*"using" + 0.004*"one" + '
  '0.003*"number" + 0.003*"problem"'),
 (1,
  '0.008*"model" + 0.007*"learning" + 0.006*"data" + 0.005*"set" + '
  '0.004*"function" + 0.004*"using" + 0.004*"one" + 0.004*"models" + '
  '0.004*"two" + 0.004*"algorithm"'),
 (2,
  '0.008*"learning" + 0.006*"function" + 0.005*"model" + 0.004*"data" + '
  '0.004*"algorithm" + 0.004*"problem" + 0.004*"one" + 0.003*"using" + '
  '0.003*"two" + 0.003*"optimization"'),
 (3,
  '0.007*"data" + 0.005*"model" + 0.005*"learning" + 0.005*"algorithm" + '
  '0.004*"models" + 0.004*"using" + 0.004*"figure" + 0.003*"set" + 0.003*"one" '
  '+ 0.003*"log"'),
 (4,
  '0.006*"learning" + 0.005*"model" + 0.005*"data" + 0.005*"function" + '
  '0.004*"network" + 0.004*"one" + 0.004*"using" + 0.003*"set" + '
  '0.003*"feature" + 0.003*"networks"'),
 (5,
  '0.007*"model" + 0.005*"algorithm" + 0.005*"learning" + 0.005*

In [ ]:
pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#import pyLDAvis.gensim

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()


LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

pyLDAvis.display(LDAvis_prepared)

## (2) (10 points) Generate K topics by using LSA, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

In [ ]:
# Write your code here

import pandas as pd

# load data
df = pd.read_csv('Musical_instruments_reviews.csv', usecols=['reviewerID', 'reviewText'])
df.head()




,reviewerID,reviewText
0,A2IBPI20UZIR0U,"Not much to write about here, but it does exac..."
1,A14VAT5EAX3D9S,The product does exactly as it should and is q...
2,A195EZSQDW3E21,The primary job of this device is to block the...
3,A2C00NNG1ZQQG2,Nice windscreen protects my MXL mic and preven...
4,A94QU4C90B1AX,This pop filter is great. It looks and perform...


In [ ]:
print(df.shape)
df = df.dropna()
print(df.shape)

(10261, 2)
(10254, 2)


In [ ]:
from gensim.parsing.preprocessing import remove_stopwords, strip_punctuation \
                                        , preprocess_string, strip_short, stem_text

# preprocess given text
def preprocess(text):
    
    # clean text based on given filters
    CUSTOM_FILTERS = [lambda x: x.lower(), 
                                remove_stopwords, 
                                strip_punctuation, 
                                strip_short, 
                                stem_text]
    text = preprocess_string(text, CUSTOM_FILTERS)
    
    return text

# apply function to all reviews 
df['Text (Clean)'] = df['reviewText'].apply(lambda x: preprocess(x))

In [ ]:
from gensim import corpora

# create a dictionary with the corpus
corpus = df['Text (Clean)']
dictionary = corpora.Dictionary(corpus)

# convert corpus into a bag of words
bow = [dictionary.doc2bow(text) for text in corpus]

In [ ]:
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

# find the coherence score with a different number of topics
for i in range(2,11):
    lsi = LsiModel(bow, num_topics=i, id2word=dictionary)
    coherence_model = CoherenceModel(model=lsi, texts=df['Text (Clean)'], dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()
    print('Coherence score with {} clusters: {}'.format(i, coherence_score))

Coherence score with 2 clusters: 0.48274890936815257
Coherence score with 3 clusters: 0.4391170417154548
Coherence score with 4 clusters: 0.44412679792352866
Coherence score with 5 clusters: 0.4192225238977871
Coherence score with 6 clusters: 0.40127792155066566
Coherence score with 7 clusters: 0.3729040899537311
Coherence score with 8 clusters: 0.414798918199683
Coherence score with 9 clusters: 0.3738069457120302
Coherence score with 10 clusters: 0.3633579360541741


In [ ]:
lsi = LsiModel(bow, num_topics=10, id2word=dictionary)

for topic_num, words in lsi.print_topics(num_words=5):
    print('Words in {}: {}.'.format(topic_num, words))

Words in 0: 0.329*"sound" + 0.314*"guitar" + 0.242*"string" + 0.232*"pedal" + 0.217*"amp".
Words in 1: 0.584*"string" + -0.428*"pedal" + 0.380*"guitar" + -0.312*"amp" + 0.161*"tune".
Words in 2: 0.529*"string" + 0.367*"pedal" + -0.261*"mic" + -0.190*"record" + 0.187*"tone".
Words in 3: 0.641*"pedal" + -0.520*"amp" + -0.306*"sound" + 0.178*"guitar" + -0.119*"mic".
Words in 4: 0.641*"guitar" + 0.351*"amp" + -0.342*"string" + -0.269*"mic" + -0.192*"record".
Words in 5: -0.460*"tuner" + 0.382*"sound" + -0.372*"tune" + -0.309*"amp" + 0.246*"guitar".
Words in 6: -0.848*"pick" + 0.214*"guitar" + -0.172*"plai" + -0.155*"like" + 0.119*"sound".
Words in 7: -0.426*"sound" + 0.384*"strap" + -0.358*"tuner" + 0.350*"cabl" + -0.308*"tune".
Words in 8: -0.450*"cabl" + 0.436*"stand" + 0.400*"mic" + -0.324*"sound" + -0.231*"good".
Words in 9: 0.670*"strap" + -0.354*"cabl" + -0.273*"guitar" + 0.181*"sound" + 0.181*"like".


## (3) (10 points) Generate K topics by using  lda2vec, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here:

https://nbviewer.org/github/cemoody/lda2vec/blob/master/examples/twenty_newsgroups/lda2vec/lda2vec.ipynb

In [ ]:
!pip install lda2vec

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Write your code here
from lda2vec import preprocess, Corpus
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

try:
    import seaborn
except:
    pass

ImportError: ignored

In [ ]:
import pyLDAvis
pyLDAvis.enable_notebook()

In [ ]:
npz = np.load(open('topics.pyldavis.npz', 'r'))
dat = {k: v for (k, v) in npz.iteritems()}
dat['vocab'] = dat['vocab'].tolist()


top_n = 10
topic_to_topwords = {}
for j, topic_to_word in enumerate(dat['topic_term_dists']):
    top = np.argsort(topic_to_word)[::-1][:top_n]
    msg = 'Topic %i '  % j
    top_words = [dat['vocab'][i].strip()[:35] for i in top]
    msg += ' '.join(top_words)
    print(msg)
    topic_to_topwords[j] = top_words

FileNotFoundError: ignored

## (4) (10 points) Generate K topics by using BERTopic, the number of topics K should be decided by the coherence score, then summarize what are the topics. You may refer the code here: 

https://colab.research.google.com/drive/1FieRA9fLdkQEGDIMYl0I3MCjSUKVF8C-?usp=sharing

In [ ]:
# Write your code here


from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('/content/tokyo_2020_tweets.csv')
df = df.dropna()
df = df.sample(20000, random_state=42)
df.head()

<ipython-input-84-cf17a970fe83>:5: DtypeWarning:

Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,15) have mixed types. Specify dtype option on import or set low_memory=False.



,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
295003,1419957058938486804,Australia in Belgium 🇦🇺🇪🇺🇧🇪🇱🇺,"Brussels, Belgium",Australian Mission to the European Union and N...,2014-11-26 15:52:22,3766,3226,4622,True,2021-07-27 09:45:19,"Congrats to 🇦🇺, 🇧🇪, and 🇱🇺 for our achievement...",['Tokyo2020'],Twitter Web App,0.0,0.0,False
63678,1419307203933229065,念真 Nian Zhen,"Enshi, Hubei, China.",一个中国\n\nChina institute of contemporary intern...,2017-06-30 20:07:29,7788,87.0,38102.0,False,2021-07-25 14:43:01,Reality Vs New York Times\n#To...,['Tokyo2020'],Twitter for Android,12.0,57.0,False
101824,1419592988833947650,Indonesia Badminton,Indonesia,Indonesia Badminton Supporter • Click to follo...,2009-11-07 23:26:53,55711,451,2081,False,2021-07-26 09:38:37,[Live Score] #Tokyo2020 \n\nWD Group A : \n\...,['Tokyo2020'],Twitter for iPhone,0.0,4.0,False
60985,1419311331782447108,FirstSportz,New Delhi,Official Twitter handle of Firstsportz. \nFoll...,2019-11-26 14:14:55,478,258.0,4053.0,False,2021-07-25 14:59:25,Tennis at Tokyo Olympics: Stefanos #Tsitsipas ...,['Tsitsipas'],Twitter for Android,0.0,0.0,False
47225,1419235863100370946,Kurdistan 24 English,"Erbil, Kurdistan",The English service of Kurdistan's leading new...,2015-07-06 08:32:11,139803,5.0,7.0,True,2021-07-25 09:59:32,The 41-year-old gold medalist is an ethnic Kur...,"['Tokyo2020', 'Olympics']",Twitter Web App,3.0,9.0,False


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

def clean_text(x):
  x = str(x)
  x = x.lower()
  x = re.sub(r'#[A-Za-z0-9]*', ' ', x)
  x = re.sub(r'https*://.*', ' ', x)
  x = re.sub(r'@[A-Za-z0-9]+', ' ', x)
  tokens = word_tokenize(x)
  x = ' '.join([w for w in tokens if not w.lower() in stop_words])
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~“…”’'), ' ', x)
  x = re.sub(r'\d+', ' ', x)
  x = re.sub(r'\n+', ' ', x)
  x = re.sub(r'\s{2,}', ' ', x)
  return x


df['clean_text'] = df.text.apply(clean_text)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet,clean_text
295003,1419957058938486804,Australia in Belgium 🇦🇺🇪🇺🇧🇪🇱🇺,"Brussels, Belgium",Australian Mission to the European Union and N...,2014-11-26 15:52:22,3766,3226,4622,True,2021-07-27 09:45:19,"Congrats to 🇦🇺, 🇧🇪, and 🇱🇺 for our achievement...",['Tokyo2020'],Twitter Web App,0.0,0.0,False,congrats 🇦🇺 🇧🇪 🇱🇺 achievements far olympics 👏👏👏
63678,1419307203933229065,念真 Nian Zhen,"Enshi, Hubei, China.",一个中国\n\nChina institute of contemporary intern...,2017-06-30 20:07:29,7788,87.0,38102.0,False,2021-07-25 14:43:01,Reality Vs New York Times\n#To...,['Tokyo2020'],Twitter for Android,12.0,57.0,False,reality vs new york times
101824,1419592988833947650,Indonesia Badminton,Indonesia,Indonesia Badminton Supporter • Click to follo...,2009-11-07 23:26:53,55711,451,2081,False,2021-07-26 09:38:37,[Live Score] #Tokyo2020 \n\nWD Group A : \n\...,['Tokyo2020'],Twitter for iPhone,0.0,4.0,False,live score wd group greysia polii apriyani ra...
60985,1419311331782447108,FirstSportz,New Delhi,Official Twitter handle of Firstsportz. \nFoll...,2019-11-26 14:14:55,478,258.0,4053.0,False,2021-07-25 14:59:25,Tennis at Tokyo Olympics: Stefanos #Tsitsipas ...,['Tsitsipas'],Twitter for Android,0.0,0.0,False,tennis tokyo olympics stefanos becomes first g...
47225,1419235863100370946,Kurdistan 24 English,"Erbil, Kurdistan",The English service of Kurdistan's leading new...,2015-07-06 08:32:11,139803,5.0,7.0,True,2021-07-25 09:59:32,The 41-year-old gold medalist is an ethnic Kur...,"['Tokyo2020', 'Olympics']",Twitter Web App,3.0,9.0,False,year old gold medalist ethnic kurd western ir...


In [ ]:
!pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from bertopic import BERTopic
tweets = df.clean_text.to_list()
timestamp = df.date.to_list()

topic_model = BERTopic(language="english")
topics, probs = topic_model.fit_transform(tweets)

topic_model.get_topic_info()

,Topic,Count,Name
0,-1,6516,-1_athletes_medals_day_nt
1,0,353,0_olympics_olympic_games_olympians
2,1,252,1_group_women_stage_soccer
3,2,245,2_garay_femenino_tandara_de
4,3,207,3_air_pistol_saurabh_manu
...,...,...,...
351,350,10,350_schooled_meet_hello_cournoyer
352,351,10,351_kj_views_follower_push
353,352,10,352_yeah_shiiiit_hooray_yeahh
354,353,10,353_luka_aivi_bearcat_luik


In [ ]:
topic_model.get_topic_info(0)

,Topic,Count,Name
0,0,353,0_olympics_olympic_games_olympians


In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(top_n_topics=12, n_words = 10, width = 350, height = 350)

## (5) (10 extra points) Compare the results generated by the four topic modeling algorithms, which one is better? You should explain the reasons in details.

# Write your answer here (no code needed for this question)
The results of LDA are as follows: '0.007"data" + 0.006"learning" + 0.005"model" + 0.005"algorithm" + ' '0.004"using" + 0.004"time" + 0.004"set" + 0.003"two" + 0.003"neural" + ' '0.003"function"

The results of LSA are as follows: 0.329"sound" + 0.314"guitar" + 0.242"string" + 0.232"pedal" + 0.217*"amp".

The results of lda2vec are as follows: topic 00 which has top words x11r5, xv, window, xterm, server, motif, font, xlib, //, sunos

The results of Bertopic are as follows: olympics_olympic_games_athletes_olympians_watching sports_summer

Bertopic results are more easily interpretable.